In [14]:
import json
import pandas as pd
import logging
import os
import shutil
import re
import filecmp
import csv
import webbrowser
from datetime import datetime, timedelta
from O365 import Account, FileSystemTokenBackend, Message
from oauth2client.service_account import ServiceAccountCredentials
import time
import numpy as np
import textwrap

class OutlookEmailer:

    def __init__(self, creds_path):
        """
        Initialize the OutlookEmailer by loading credentials from the given file.
        
        :param creds_path: Path to the credentials JSON file.
        """
        self.creds_dict = self._load_creds(creds_path)

    def _load_creds(self, path):
        """
        Load credentials from the provided file path.
        
        :param path: Path to the credentials JSON file.
        :return: A dictionary containing the credentials.
        """
        with open(path) as file:
            creds_data = json.load(file)

        return {
            'description': creds_data["outlook_description"],
            'client_secret': creds_data["outlook_client_secret"],
            'application_id': creds_data["outlook_application_id"],
            'shared_mailbox': creds_data["outlook_shared_mailbox"],
            'personal_mailbox': creds_data["outlook_personal_mailbox"],
            'tenant_id': creds_data["outlook_tenant_id"]
        }
    
    def _fold_lines(self, ics_content):
        """
        Fold long lines according to ICS file format rules.
        Lines longer than 75 characters should be wrapped with a space on the following line.
        """
        lines = ics_content.splitlines()
        folded_lines = []
        
        for line in lines:
            while len(line) > 75:
                # Split the line at 75 characters and fold it to the next line
                folded_lines.append(line[:75])
                line = ' ' + line[75:]
            folded_lines.append(line)
        
        return '\r\n'.join(folded_lines)


    def get_creds(self, creds_file_path):
        """
        Get credentials by loading them from a specified path.
        
        :param creds_file_path: Path to the credentials JSON file.
        :return: A dictionary containing the credentials.
        """
        return self._load_creds(creds_file_path)

    def load_attachments(self, folder_path='attachments'):
        files = os.listdir(folder_path)
        return {os.path.splitext(file)[0]: os.path.join(folder_path, file) for file in files}

    def authenticate_outlook(self, mailbox='personal'):
        credentials = (self.creds_dict['application_id'], self.creds_dict['client_secret'])
        token_backend = FileSystemTokenBackend(token_path='.', token_filename='o365_token.txt')

        # Determine which mailbox to use
        if mailbox == 'shared':
            main_resource = self.creds_dict['shared_mailbox']
        else:
            main_resource = self.creds_dict['personal_mailbox']
        
        # Create an Account object with the chosen mailbox
        account = Account(credentials, token_backend=token_backend, main_resource=main_resource)

        # Add 'Mail.Send' to the scopes
        scopes = ['User.Read', 'Mail.Read', 'Mail.Send', 'Mail.Read.Shared', 'Mail.Send.Shared']

        # Try to load the token from the file
        if not account.is_authenticated:
            # Get the authorization URL
            url, _ = account.con.get_authorization_url(requested_scopes=scopes)

            # Open the URL in a web browser
            webbrowser.open(url)

            # Prompt the user to manually copy the authentication URL
            print("Please manually copy the authentication URL from the web browser and paste it below.")
            auth_url = input("Authentication URL: ")

            # Authenticate with the provided URL and scopes
            if account.authenticate(url=auth_url, scopes=scopes):
                # Save the token to the file
                "Authentication successful. Token saved to 'o365_token.txt'."
            else:
                print("Authentication failed.")
                return None

        return account


    def send_email_from_outlook(self, sender, recepients, subject, email_body, attachments_paths, code=None):

        account = self.authenticate_outlook(self.creds_dict)

        if account is not None and account.is_authenticated:
            print('Outlook email authentication successful.')

            m = account.new_message()
            m.sender.address = sender  # The shared mailbox
            m.to.add(recepients)
            m.subject = subject

            # Check if attachments_paths is not None
            if attachments_paths is not None:
                # Loop over attachments paths
                for path in attachments_paths:
                    # Skip over None values
                    if path is not None:
                        # Add the attachment
                        m.attachments.add(path)

            m.body = email_body
            m.send()

            print(f"Email {code} from {sender} to {recepients} sent successfully!")
            return True
        else:
            print("Failed to send email. Authentication unsuccessful.")
            return False

        #TODO: Figure out when authentication expires, and how to make it last longer?

    def create_ics_attachment(self, subject, description, start_time, end_time, location):
        # Generate the ICS content as a string
        ics_content = textwrap.dedent(f"""\
    BEGIN:VCALENDAR
    VERSION:2.0
    PRODID:-//Your Organization//NONSGML v1.0//EN
    BEGIN:VEVENT
    UID:{datetime.now().strftime('%Y%m%dT%H%M%SZ')}
    DTSTAMP:{datetime.now().strftime('%Y%m%dT%H%M%SZ')}
    DTSTART:{start_time.strftime('%Y%m%dT%H%M%SZ')}
    DTEND:{end_time.strftime('%Y%m%dT%H%M%SZ')}
    SUMMARY:{subject}
    DESCRIPTION:{description}
    LOCATION:{location}
    END:VEVENT
    END:VCALENDAR
    """)

        # Ensure lines don't exceed 75 characters for ICS format
        ics_content = self._fold_lines(ics_content)

        # Write ICS content to a file
        ics_file_path = "calendarInvite.ics"
        with open(ics_file_path, 'w') as f:
            f.write(ics_content)

        return ics_file_path

    
    def is_valid_email(self, email):
        email = email.lower()  
        email_regex = r'^\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b$'
        return re.match(email_regex, email) is not None
    

    def get_email_template(self, email_code, templates_df):
        mask = templates_df["EmailCode"].astype(str) == email_code
        email_body = templates_df[mask]['EmailBody'].iloc[0]
        subject = templates_df[mask]['Subject'].iloc[0]
        escaped_email_body = re.sub(r"{(?!\w)", "{{", email_body)
        escaped_email_body = re.sub(r"(?<=\W)}", "}}", escaped_email_body)
        return subject, escaped_email_body

    def send_reminders(self, email_list, email_code, templates_df):
        if not email_list:
            return

        subject, email_body = self.get_email_template(email_code, templates_df)
        
        for email in email_list:
            self.send_email_from_outlook(self.creds_dict, SENDER_EMAIL, [email], subject, email_body, attachments_paths=None, code=email_code)
            self.google_sheet.attempt_request(self.google_sheet.update_column_value, 'Journalling Sign Ups', 5, "Email", email)
            self.google_sheet.attempt_request(self.google_sheet.update_column_value, 'Journalling Sign Ups', 5, "EmailCode", email_code)
            self.google_sheet.attempt_request(self.google_sheet.update_column_value, 'Journalling Sign Ups', 5, "SentAt", str(dt.datetime.now()))

    def generate_src_tag(self, file_paths):
        src_tags = {}
        
        for file_path in file_paths:
            # Encode the image in base64
            encoded_image = self.encode_image_base64(file_path)
            
            # Extract the file extension (without the dot)
            file_extension = os.path.splitext(file_path)[1][1:]
            
            # Generate the filename (without the extension)
            filename = os.path.splitext(os.path.basename(file_path))[0]
            
            # Generate the src tag
            src_tag = f'<img src="data:image/{file_extension};base64,{encoded_image}" alt="{filename} Image" width="200">'
            
            # Append the src tag to the dictionary
            src_tags[filename] = src_tag
        
        return src_tags

    def encode_image_base64(image_path):
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        return encoded_string
    

    def find_file_with_extension(self, directory, filename_prefix):
        # List all files in the given directory
        files = os.listdir(directory)
        
        # Search for a file that starts with the given filename_prefix
        for file in files:
            if file.startswith(filename_prefix):
                return file  # return the full filename including its extension
        return None  # return None if no matching file was found

    def read_emails_from_outlook(self, folder_name='Inbox', query=None, limit=10, mailbox='personal'):
    # Authenticate with the specified mailbox
        account = self.authenticate_outlook(mailbox=mailbox)
        if not account or not account.is_authenticated:
            print("! Failed to authenticate Outlook account for reading emails.")
            return None

        # Get the mailbox
        if mailbox == 'personal':
            mailbox = self.creds_dict["personal_mailbox"]
        elif mailbox == 'shared':
            mailbox = self.creds_dict["shared_mailbox"]

        mailbox = account.mailbox(mailbox)

        # Get the folder to read emails from (default is 'Inbox')
        folder = mailbox.get_folder(folder_name) if folder_name != 'Inbox' else mailbox.inbox_folder()

        # Query emails
        try:
            emails = folder.get_messages(limit=limit, query=query)
        except Exception as e:
            print(f"Error retrieving emails: {e}")
            return None

        email_data = []
        for message in emails:
            email_info = {
                'subject': message.subject,
                'from': message.sender.address,
                'to': [recipient.address for recipient in message.to],
                'received': message.received,
                'body': message.body,
                'attachments': [attachment.name for attachment in message.attachments]
            }
            email_data.append(email_info)

            # Attempt to mark emails as read
            # try:
            #     message.mark_as_read()
            # except Exception as e:
            #     print(f"Error marking message as read (message subject: {message.subject}): {e}")  # Print the specific error encountered
            #     continue

        return email_data

class Scheduler:

    def __init__(self, paths, backup_limit=60, overwrite_local=True):

        self.paths = self.__load_paths(paths)
        self.__reload_dfs()
        
        self.local_dir = self.paths['local_dir']
        self.backup_dir = self.paths['backup_dir']
        self.log_file = self.paths['log_file']

        self.missing_schedule_checked = False
        
        self.emailer = OutlookEmailer(self.paths['creds'])

        self.date_time_now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        self.date_time_now_fileformat = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.backup_limit = backup_limit
        self.overwrite_local = overwrite_local
        
        self.dates = ['V1_Date', 'V2_Date', 'V3_Date', 'V4_Date', 'V5_Date', 'V6_Date', 'V7_Date']
        self.times = ['V1_Time', 'V2_Time', 'V3_Time', 'V4_Time', 'V5_Time', 'V6_Time', 'V7_Time']

        logging.basicConfig(
            filename=self.log_file, 
            level=logging.INFO, 
            format='%(asctime)s - %(levelname)s - %(message)s', 
            datefmt='%Y-%m-%d %H:%M:%S'
        )

        # load dataframes
    
    def __reload_dfs(self):
        # local
        self.df_vs_l = pd.read_csv(self.paths['visit_schedule_local'])
        self.df_es_l = pd.read_csv(self.paths['email_schedule_local'])
        self.df_et_l = pd.read_csv(self.paths['email_templates_local'])
        self.df_el_l = pd.read_csv(self.paths['email_log_local'])
        # streamlit
        self.df_vs_s = pd.read_csv(self.paths['visit_schedule_streamlit'])
        self.df_et_s = pd.read_csv(self.paths['email_templates_streamlit'])
    
    def __load_paths(self, paths):
        with open(paths, 'r') as f:
            paths = json.load(f)
        return paths

    def backup_local_csvs(self):
        print("# Backing up local CSVs.")
        if not os.path.exists(self.backup_dir):
            os.makedirs(self.backup_dir)
            
        log_details = []
        
        for filename in os.listdir(self.local_dir):
            if filename.endswith('.csv'):
                subfolder_name = filename.rsplit('.', 1)[0]
                subfolder_path = os.path.join(self.backup_dir, subfolder_name)
                
                if not os.path.exists(subfolder_path):
                    os.makedirs(subfolder_path)

                source_file_path = os.path.join(self.local_dir, filename)

                # Get the most recent backup file in the subfolder (if any)
                existing_backups = [f for f in os.listdir(subfolder_path) if f.endswith('.csv')]
                existing_backups.sort(reverse=True)  # Sort backups by date

                most_recent_backup = existing_backups[0] if existing_backups else None
                most_recent_backup_path = os.path.join(subfolder_path, most_recent_backup) if most_recent_backup else None

                # Compare the current file with the most recent backup (if any)
                if most_recent_backup_path and filecmp.cmp(source_file_path, most_recent_backup_path, shallow=False):
                    log_details.append(f"    No changes detected in {filename}, skipping backup.")
                else:
                    backup_filename = f"{subfolder_name}_{self.date_time_now_fileformat}.csv"
                    backup_file_path = os.path.join(subfolder_path, backup_filename)
                    if self.overwrite_local:
                        shutil.copy2(source_file_path, backup_file_path)
                    log_details.append(f"    * Backed up {filename}.")

        # Write log details to the log file
        with open(self.log_file, 'a') as log:
            for detail in log_details:
                log.write(detail + '\n')
                print(detail)
        
        print('    > Backup complete.')

    def clear_old_backups(self):
        print("# Clearing old backups.")
        print(f"    Starting to scan {self.backup_dir}...")  # Debugging print
        pattern = re.compile(r"(\d{8})_(\d{6})\.csv$")
        for foldername, _, filenames in os.walk(self.backup_dir):
            print(f"    Scanning folder: {foldername}")  # Debugging print
            for filename in filenames:
                match = pattern.search(filename)
                if not match:
                    print(f"    Skipping: {filename}")  # Debugging print for filenames that don't match the pattern
                    continue
                
                date_str, time_str = match.groups()
                
                # Convert date and time strings to a datetime object
                file_date = datetime.strptime(date_str + time_str, "%Y%m%d%H%M%S")
                self.dtm = datetime.strptime(self.date_time_now_fileformat, "%Y%m%d_%H%M%S")
                
                # Check file age
                if self.backup_limit == 0:
                    # remove all files
                    file_path = os.path.join(foldername, filename)
                    try:
                        os.remove(file_path)
                        print(f"    Removed {file_path}")
                    except Exception as e:
                        print(f"    Unable to remove {file_path}. Reason: {e}")
                elif (self.dtm - file_date).days > self.backup_limit:
                    file_path = os.path.join(foldername, filename)
                    try:
                        os.remove(file_path)
                        print(f"    Removed {file_path}")
                    except Exception as e:
                        print(f"    Unable to remove {file_path}. Reason: {e}")


        print(f"    > Finished scanning {self.backup_dir}.")  # Debugging print

    def check_for_changes(self):
        print("# Checking for changes between local and streamlit files.")
        pairs = []
        grouped_paths = {}
        changes_dict = {}

        # Step 1: Group paths by their common prefix
        for key, path in self.paths.items():
            # if key does not contain an underscore, skip
            if '_' not in key:
                continue
            prefix, suffix = key.rsplit('_', 1)
            if prefix not in grouped_paths:
                grouped_paths[prefix] = []
            grouped_paths[prefix].append((suffix, path))

        # Step 2: Create pairs from the grouped paths
        for prefix, path_list in grouped_paths.items():
            if len(path_list) > 1:
                for i in range(len(path_list)):
                    for j in range(i + 1, len(path_list)):
                        suffix1, path1 = path_list[i]
                        suffix2, path2 = path_list[j]
                        pairs.append(((suffix1, path1), (suffix2, path2)))

        # Step 3: Compare files in each pair and check for changes
        for pair in pairs:
            (suffix1, path1), (suffix2, path2) = pair
            filename = path1.split('/')[-1]
            
            # Compare the files and capture changes
            if filecmp.cmp(path1, path2, shallow=False):
                print(f"    No updates in {filename}.")
            else:
                print(f"    ! Changes detected in {filename}.")
                self.process_changes(filename, path1, path2)
                if self.overwrite_local:
                    shutil.copy2(path2, path1)
                    print(f"    * Updated {filename}.")
        
        print('    > Check for changes completed.')
        return changes_dict


    def process_changes(self, filename, path1, path2):
        if filename == '1_visit_schedule.csv':
            self.process_visit_schedule_changes(filename, path1, path2)
                
    
    def process_visit_schedule_changes(self, filename, path1, path2):
        pids_needing_new_schedule = []
        
        for index, row in self.df_vs_s.iterrows():
            pid = row['ParticipantID']

            # NEW PARTICIPANT
            if not pid in self.df_vs_l['ParticipantID'].values:
                print("        New participant registered:", pid)
                pids_needing_new_schedule.append(pid)

            # CHANGES TO EXISTING PARTICIPANT
            else:
                row_l = self.df_vs_l[self.df_vs_l['ParticipantID'] == pid]
                row_s = self.df_vs_s[self.df_vs_s['ParticipantID'] == pid]
                if not row_l.equals(row_s):
                    print("        Visit information updated for participant:", pid)
                    pids_needing_new_schedule.append(pid)

        if pids_needing_new_schedule:
            print("    > Generating new email schedule for updated participants:", pids_needing_new_schedule)
            self.generate_email_schedule(pids_needing_new_schedule)


    def check_missing_schedule(self): 
        
        print("# Checking for missing schedules.")
        pids_needing_new_schedule = []               
        # PARTICIPANT WITH MISSING EMAIL SCHEDULE
        for pid in self.df_vs_l['ParticipantID'].values:
            if pid not in pids_needing_new_schedule:
                if not pid in self.df_es_l['ParticipantID'].values:
                    print("    Participant missing email schedule:", pid)
                    pids_needing_new_schedule.append(pid)
            
        if len(pids_needing_new_schedule) > 0:
            print("    > Generating new email schedule for participants:", pids_needing_new_schedule)
            self.generate_email_schedule(pids_needing_new_schedule)

        self.missing_schedule_checked = True

        if not len(pids_needing_new_schedule) > 0:
            print("    > No missing schedules found.")

        return pids_needing_new_schedule

    def generate_email_schedule(self, pids):
        email_templates = {}
        for index, row in self.df_et_l.iterrows():
            email_templates[row['EmailCode']] = {
                'Subject': row['Subject'],
                'EmailBody': row['EmailBody'],
                'Offset': row['Offset'],
                'VisitNumber': row['VisitNumber'],
            }
        
        print("    Loaded email codes:", email_templates.keys())

        rows_to_add = []
        for pid in pids:
            # Remove future scheduled emails for the participant before creating a new schedule
            self.remove_future_scheduled_emails(pid)

            # Generate new email schedule only if the EmailCode is not already present for this participant
            participant_row = self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == pid]
            existing_email_codes = self.df_es_l[self.df_es_l['ParticipantID'] == pid]['EmailCode'].tolist()

            for email_code, info in email_templates.items():
                if email_code not in existing_email_codes:
                    rows_to_add.append({
                        'ParticipantID': pid,
                        'EmailCode': email_code,
                        'ScheduledDate': self.calculate_scheduled_date(pid, info['VisitNumber'], info['Offset']),
                        'UpdatedAt': self.date_time_now,
                    })

        new_rows = pd.DataFrame(rows_to_add)
        self.df_es_l = pd.concat([self.df_es_l, new_rows], axis=0).reset_index(drop=True)
        if self.overwrite_local:
            self.df_es_l.to_csv(self.paths['email_schedule_local'], index=False)
            self.__reload_dfs()
            print("      * Added new email schedule for participants: ", pids)

        # Check for existing participants to update their schedule with new templates
        self.update_existing_participants_schedule(email_templates)

    def update_existing_participants_schedule(self, email_templates):
        """
        Update the email schedule for existing participants if there are new email templates added.
        """
        print("    Checking for new email templates to update existing schedules...")
        
        # Get all participants who already have a schedule
        participants = self.df_es_l['ParticipantID'].unique()
        
        rows_to_add = []
        for pid in participants:
            existing_email_codes = self.df_es_l[self.df_es_l['ParticipantID'] == pid]['EmailCode'].tolist()
            
            # Check if any email templates are missing in the existing schedule
            for email_code, info in email_templates.items():
                if email_code not in existing_email_codes:
                    # Only add if the email is not already scheduled
                    rows_to_add.append({
                        'ParticipantID': pid,
                        'EmailCode': email_code,
                        'ScheduledDate': self.calculate_scheduled_date(pid, info['VisitNumber'], info['Offset']),
                        'UpdatedAt': self.date_time_now,
                    })

        # If new rows are added, update the email schedule
        if rows_to_add:
            new_rows = pd.DataFrame(rows_to_add)
            self.df_es_l = pd.concat([self.df_es_l, new_rows], axis=0).reset_index(drop=True)
            if self.overwrite_local:
                self.df_es_l.to_csv(self.paths['email_schedule_local'], index=False)
                self.__reload_dfs()
            print("    * Updated existing participants' email schedules with new templates.")


    def calculate_scheduled_date(self, pid, visit_code, offset):
        visit_date = self.df_vs_s.loc[self.df_vs_s['ParticipantID'] == pid, f"{visit_code}_Date"].values[0]
        visit_date = datetime.strptime(visit_date, "%Y-%m-%d")       
        scheduled_date = visit_date + timedelta(days=offset)
        scheduled_date = scheduled_date.strftime("%Y-%m-%d")
        return scheduled_date
    
    # THESE ARE NEW AND NEED TO BE CHECKED

    def check_emails_to_send_today(self):
        print("# Checking for emails to be sent today.")
        today = datetime.now().strftime('%Y-%m-%d')
        emails_to_send = self.df_es_l[self.df_es_l['ScheduledDate'] == today]
        
        if emails_to_send.empty:
            print("    > No emails to be sent today.")
            return []
        
        print(f"    > Found {len(emails_to_send)} emails to send today.")
        # print the emails 
        for _, row in emails_to_send.iterrows():
            print(f"        Participant {row['ParticipantID']} scheduled to receive email {row['EmailCode']} today.")
        return emails_to_send
    
    def create_email_dict(self, emails_to_send_today):
        email_dict = {}
        for _, row in emails_to_send_today.iterrows():
            # check if participant is active
            self.__reload_dfs()
            if self.df_vs_l.loc[
                self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'Active'].empty or \
                self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'Active'].values[0] not in ['True', True]:
                print(f"        Participant {row['ParticipantID']} is not active. Skipping email.")
            # check if email was already sent
            elif not self.df_el_l.loc[(self.df_el_l['ParticipantID'] == row['ParticipantID']) & (self.df_el_l['EmailCode'] == row['EmailCode'])].empty:
                print(f"        Email {row['EmailCode']} for participant {row['ParticipantID']} was already sent. Skipping.")
            else:
                # Check if the participant already exists in the dictionary
                if row['ParticipantID'] not in email_dict:
                    email_dict[row['ParticipantID']] = {}
                
                # Add the email details to the participant's dictionary
                email_dict[row['ParticipantID']][row['EmailCode']] = { 
                    'ParticipantID': row['ParticipantID'],
                    'EmailCode': row['EmailCode'],
                    'ScheduledDate': row['ScheduledDate'],
                    'UpdatedAt': row['UpdatedAt'],
                    'Email': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'Email'].values[0],
                    'FirstName': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'FirstName'].values[0],
                    'Surname': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'Surname'].values[0],
                    'Subject': self.df_et_l.loc[self.df_et_l['EmailCode'] == row['EmailCode'], 'Subject'].values[0],
                    'EmailBody': self.df_et_l.loc[self.df_et_l['EmailCode'] == row['EmailCode'], 'EmailBody'].values[0],
                    'V1_Date': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V1_Date'].values[0],
                    'V2_Date': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V2_Date'].values[0],
                    'V3_Date': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V3_Date'].values[0],
                    'V4_Date': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V4_Date'].values[0],
                    'V5_Date': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V5_Date'].values[0],
                    'V6_Date': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V6_Date'].values[0],
                    'V7_Date': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V7_Date'].values[0],
                    'V1_Time': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V1_Time'].values[0],
                    'V2_Time': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V2_Time'].values[0],
                    'V3_Time': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V3_Time'].values[0],
                    'V4_Time': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V4_Time'].values[0],
                    'V5_Time': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V5_Time'].values[0],
                    'V6_Time': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V6_Time'].values[0],
                    'V7_Time': self.df_vs_l.loc[self.df_vs_l['ParticipantID'] == row['ParticipantID'], 'V7_Time'].values[0],
                }

        return email_dict

    
    def check_attachments(self, email_code):

        # Find value of column Attachments in df_et_l
        attachment = self.df_et_l.loc[self.df_et_l['EmailCode'] == email_code, 'Attachments'].values[0]

        # Check if the attachment is not None, 'None', or an empty string
        if not pd.isna(attachment) and attachment not in ['False', False, 'None', 'none', 'NONE', '']:
            # Check if it's a list of attachments (comma-separated)
            if ',' in attachment:
                attachment = [a.strip() for a in attachment.split(',')]  # Split and strip whitespace
            else:
                attachment = [attachment.strip()]  # Convert single attachment to list
            # Prepend the path to each attachment
            attachment = [os.path.join(self.paths['attachments'], a) for a in attachment]
        else:
            attachment = None

        return attachment
    
    def check_calendar_event(self, email_code):
        cal_event = self.df_et_l.loc[self.df_et_l['EmailCode'] == email_code, 'CalendarEvent'].values[0]
        if cal_event in [0, '0']:
            cal_event = False
        else:
            cal_event = True
        
        return cal_event
    

    def format_calendar_event(self, email_data):
        # Find the corresponding visit number for the emal code
        visit_number = self.df_et_l.loc[self.df_et_l['EmailCode'] == email_data['EmailCode'], 'VisitNumber'].values[0]
        visit_number = visit_number[-1]  # Only keep the last character

        subject = f"UNITY Visit {visit_number} for {email_data['ParticipantID']}"
        description = f"UNITY Visit {visit_number} for {email_data['ParticipantID']}"
        
        # Combine date and time fields from email_data
        start_date_str = email_data[f'V{visit_number}_Date']
        start_time_str = email_data[f'V{visit_number}_Time']
        
        # Split the date and time strings into components
        year, month, day = map(int, start_date_str.split('-'))  # Split and convert to integers
        hour, minute = map(int, start_time_str.split(':'))  # Split and convert to integers

        # Create a datetime object using the parsed components
        start_time = datetime(year, month, day, hour, minute)
        # Add 3 hours for the end time
        end_time = start_time + timedelta(hours=3)

        location = 'UCL'

        # Generate the ICS attachment with formatted times
        cal_attachment = self.emailer.create_ics_attachment(subject, description, start_time, end_time, location)

        return cal_attachment



    def send_email(self, pid, email_data):

        email_code = email_data['EmailCode']
        subject = email_data['Subject']
        body = email_data['EmailBody']
        escaped_email_body = re.sub(r"{(?!\w)", "{{", body)
        escaped_email_body = re.sub(r"(?<=\W)}", "}}", escaped_email_body)
        formatted_email_body = escaped_email_body.format(**email_data)
        recipient = email_data['Email']
        attachment = self.check_attachments(email_code)
        calendar_event = self.check_calendar_event(email_code)
        if calendar_event:
            # add to attachment
            cal_attachment = self.format_calendar_event(email_data)
            if attachment:
                attachment.append(cal_attachment)
            else:
                attachment = [cal_attachment]
                              

        status = self.emailer.send_email_from_outlook(sender=self.paths['email_sender'],
                                            recepients=[recipient],
                                            subject=subject,
                                            email_body=formatted_email_body,
                                            attachments_paths=attachment,  # Add if necessary
                                            code=email_code)
        
        filtered_df = self.df_es_l.loc[
                        (self.df_es_l["ParticipantID"] == pid)
                        & (self.df_es_l["EmailCode"] == email_code),
                        "ScheduledDate",
                    ]

        scheduled_for = None
        if filtered_df.empty:
            raise ValueError(
                f"No matching records found for ParticipantID: {pid} and EmailCode: {email_code}"
            )
        else:
            scheduled_for = filtered_df.values[0]
        
        email_receipt = {
            'ParticipantID': pid,
            'EmailCode': email_code,
            'ScheduledFor': scheduled_for,
            'SentAt': self.date_time_now,
            'Status': status
        }
        time.sleep(2)
        return email_receipt
        
    
    def update_local_email_log(self, email_receipts):

        dfx = pd.DataFrame(email_receipts)
            
        # concat to df_local_el
        self.df_el_l = pd.concat([self.df_el_l, dfx], axis=0).reset_index(
            drop=True
        )
        # save to csv
        if self.overwrite_local:
            self.df_el_l.to_csv(self.paths['email_log_local'], index=False)
            for email_receipt in email_receipts:
                print(f"     * Updated local email log for participant: {email_receipt['ParticipantID']}: {email_receipt['EmailCode']}.")

    def remove_future_scheduled_emails(self, participant_id):
        """
        Remove future scheduled emails for the given participant ID.
        
        :param participant_id: The ID of the participant whose future emails should be removed.
        """
        # Get today's date
        today = datetime.now().strftime("%Y-%m-%d")
        
        # Filter out rows where the participant ID matches and the scheduled date is in the future
        self.df_es_l = self.df_es_l[~((self.df_es_l['ParticipantID'] == participant_id) & (self.df_es_l['ScheduledDate'] > today))]
        
        # Optionally save changes back to the local file
        if self.overwrite_local:
            self.df_es_l.to_csv(self.paths['email_schedule_local'], index=False)
            print(f"     * Removed future scheduled emails for participant: {participant_id}")

    
    def main(self):
        print("REPORT TIME:", self.date_time_now)
        self.clear_old_backups()
        self.backup_local_csvs()
        changes = self.check_for_changes()
        if not self.missing_schedule_checked:
            self.check_missing_schedule()
        emails_to_send_today = self.check_emails_to_send_today()

        # # NEEDS TO BE CHECKED
        email_receipts = []
        if not emails_to_send_today.empty:
            email_dict = self.create_email_dict(emails_to_send_today)
            for pid, values in email_dict.items():
                for email_code, email_data in values.items():
                    email_receipt = self.send_email(pid, email_data)
                    email_receipts.append(email_receipt)
            if email_receipts:
                self.update_local_email_log(email_receipts)


if __name__ == '__main__':
    paths = 'admin/paths.json'
    scheduler = Scheduler(paths, backup_limit=60, overwrite_local=True)
    scheduler.main()


REPORT TIME: 2024-10-17 21:07:53
# Clearing old backups.
    Starting to scan /data/jkuc/unity_scheduler/data/backup...
    Scanning folder: /data/jkuc/unity_scheduler/data/backup
    Scanning folder: /data/jkuc/unity_scheduler/data/backup/3_email_templates
    Scanning folder: /data/jkuc/unity_scheduler/data/backup/4_email_log
    Scanning folder: /data/jkuc/unity_scheduler/data/backup/2_email_schedule
    Scanning folder: /data/jkuc/unity_scheduler/data/backup/1_visit_schedule
    > Finished scanning /data/jkuc/unity_scheduler/data/backup.
# Backing up local CSVs.
    No changes detected in 4_email_log.csv, skipping backup.
    * Backed up 3_email_templates.csv.
    No changes detected in 2_email_schedule.csv, skipping backup.
    No changes detected in 1_visit_schedule.csv, skipping backup.
    > Backup complete.
# Checking for changes between local and streamlit files.
    No updates in 1_visit_schedule.csv.
    ! Changes detected in 3_email_templates.csv.
    * Updated 3_email_tem

Tested scenarios:
* Manually adding a new participant to streamlit visit schedule file. 
* Successfully copies over to local visit schedule. 
* Successfully generates email schedule.
* Successfully sends emails to newly registered participants if they should receive email today.
* Backup works. 
* It can send multiple emails to the same/different participants in one run. 
* If date is manually changed in streamlit file visit schedule, new schedule is generated. Emails scheduled for the future date are cleared from the current schedule and replaced with new emails. Emails scheduled for day in past are not cleared, because we don't want to regenerate the dates for the visit reminders that have already been happened.

TO DO: 
* What doesn't work is when a new email is added to email templates, the email schedule for already registered participants is not updated.
* Add logging / export print statements. 
* Do we want to default visit time to 3 hours or do we want this to be dynamically changing based on visit? 
* Integrate new emails template into the code.
